<a href="https://colab.research.google.com/github/eric-promptshop/FortuneFinders/blob/main/FortuneFindersDiscordNewsAutomation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Build automation that sends a daily stock market update from AI to a Discord news channel.

## Set up discord integration

### Subtask:
Set up a Discord bot, obtain its token, add it to your server, and get the channel ID for your news channel.


## Stock market data retrieval

### Subtask:
Find a reliable source for stock market data and use an API or library to fetch the data.


**Reasoning**:
Research indicates that `yfinance` is a widely used and free library for fetching historical stock data from Yahoo Finance. This library is easy to use and provides access to daily closing prices and trading volumes, which aligns with the subtask requirements. The following code will use `yfinance` to fetch data for a few major indices and stocks and store it in a pandas DataFrame.



In [3]:
!pip install yfinance

In [5]:
import yfinance as yf
import pandas as pd

# Define the tickers for the stocks and indices
tickers = ["SPY", "QQQ", "BTC-USD", "ETH-USD"]

# Create a dictionary to store the data
stock_data = {}

# Fetch the data for each ticker
for ticker in tickers:
    stock = yf.Ticker(ticker)
    # Get historical data for the last day
    hist = stock.history(period="1d")
    stock_data[ticker] = hist

# Concatenate the data into a single DataFrame
df_stock_data = pd.concat(stock_data)

# Display the data
display(df_stock_data)

,,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
,Date,,,,,,,,
SPY,2025-07-03 00:00:00-04:00,622.450012,626.280029,622.429993,625.340027,51065800,0.0,0.0,0.0
QQQ,2025-07-03 00:00:00-04:00,553.179993,557.200012,553.179993,556.219971,26443500,0.0,0.0,0.0
BTC-USD,2025-07-05 00:00:00+00:00,108019.601562,108352.476562,107870.320312,108034.445312,32925065216,0.0,0.0,NaN
ETH-USD,2025-07-05 00:00:00+00:00,2508.435791,2528.364990,2496.699463,2496.699463,10292975616,0.0,0.0,NaN


In [11]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [13]:
# Initialize the Gemini API
gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-04-17')

In [16]:
# Convert the DataFrame to a string format suitable for the model
stock_data_string = df_stock_data.to_string()

# Create a prompt for the AI model
prompt = f"""
Summarize the following stock market data for a daily update in a Discord news channel.
Format the summary exactly as shown in the example below, using the provided data:

📅 [Current Day of the Week], [Current Month] [Current Day], [Current Year] – U.S. Market Close Summary (Automation Turn [Automation Turn Number])

Major Indices:
        •       SPY [closing price], [change from open]%
        •       QQQ [closing price], [change from open]%
        •       BTC-USD [closing price], [change from open]%
        •       ETH-USD [closing price], [change from open]%

Key Drivers:
        •       [Identify key drivers from the data, e.g., price movements, volume]
        •       [Add more key drivers]

Broader Context:
        •       [Provide broader context based on the data and current market conditions]
        •       [Add more context]

⸻

✅ Summary:
[Provide an overall summary of the market activity based on the data and context]

Here is the data to summarize:
{stock_data_string}
"""

# Generate the summary using the Gemini model
response = gemini_model.generate_content(prompt)
stock_summary = response.text

# Display the generated summary
print(stock_summary)

📅 Thursday, July 3, 2025 – U.S. Market Close Summary (Automation Turn 1)

Major Indices:
        •       SPY 625.34, +0.46%
        •       QQQ 556.22, +0.55%
        •       BTC-USD 108034.45, +0.01%
        •       ETH-USD 2496.70, -0.47%

Key Drivers:
        •       Moderate positive movements in major US stock indices (SPY and QQQ).
        •       Bitcoin traded mostly flat, showing minimal change from its opening price.
        •       Ethereum experienced a slight decline.
        •       Volume data provided alongside price changes for all assets.

Broader Context:
        •       US equity markets exhibited modest gains based on the provided July 3rd data.
        •       Cryptocurrency data provided is from a later date (July 5th), showing mixed performance with Bitcoin stable and Ethereum slightly down.

⸻

✅ Summary:
Based on the provided data, US stock indices SPY and QQQ posted moderate gains on July 3rd. Bitcoin remained largely unchanged on July 5th, while Ethereum saw

In [17]:
import requests

# Replace with your actual Discord webhook URL
discord_webhook_url = "https://discord.com/api/webhooks/1353172402401574923/NeyevkVUc-bBjCGUGmJRI-169jYCPt-AF7yH8MktGx3dKLyxsir6NWuAoh4xR4kIuuFQ"

# The stock summary generated by the AI model
# Make sure the 'stock_summary' variable from the previous cell is available
# stock_summary = "Your generated stock summary here" # Uncomment and replace if needed

payload = {
    "content": stock_summary
}

try:
    response = requests.post(discord_webhook_url, json=payload)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    print("Stock market update sent to Discord successfully!")
except requests.exceptions.RequestException as e:
    print(f"Error sending message to Discord: {e}")

Stock market update sent to Discord successfully!
